In [1]:
import requests
import findspark
import pandas as pd
import numpy as np
findspark.init()
from pyspark.sql import SparkSession
from pyspark.sql.functions import*
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, BooleanType, DoubleType
from pyspark import SparkContext
from pyspark.sql.functions import monotonically_increasing_id

In [2]:
spark = SparkSession.builder.appName("Branch_App").getOrCreate()

In [3]:
df_branch = spark.read.json(r'C:\suneetha\Python_labs\Capstone_project\CreditCard_dataset\cdw_sapp_branch.json')

In [4]:
df_branch.printSchema()

root
 |-- BRANCH_CITY: string (nullable = true)
 |-- BRANCH_CODE: long (nullable = true)
 |-- BRANCH_NAME: string (nullable = true)
 |-- BRANCH_PHONE: string (nullable = true)
 |-- BRANCH_STATE: string (nullable = true)
 |-- BRANCH_STREET: string (nullable = true)
 |-- BRANCH_ZIP: long (nullable = true)
 |-- LAST_UPDATED: string (nullable = true)



In [5]:
#Replace null values in 'BRANCH_ZIP' with '999999'
df_branch = df_branch.fillna({'BRANCH_ZIP': '999999'})
df_branch.select('BRANCH_ZIP').show()

+----------+
|BRANCH_ZIP|
+----------+
|     55044|
|     60142|
|     11419|
|     32068|
|     19406|
|      7501|
|     14534|
|      6109|
|     44070|
|      8844|
|     48071|
|     32765|
|     17050|
|     11803|
|     42001|
|     19438|
|     20901|
|     55337|
|     98444|
|     17013|
+----------+
only showing top 20 rows



In [6]:
#Format 'BRANCH_PHONE'
df_branch= df_branch.withColumn('BRANCH_PHONE', regexp_replace('BRANCH_PHONE', '^(\d{3})(\d{3})(\d{4})$', '(610)$1-$2'))
df_branch.show()

+-----------------+-----------+------------+------------+------------+-------------------+----------+--------------------+
|      BRANCH_CITY|BRANCH_CODE| BRANCH_NAME|BRANCH_PHONE|BRANCH_STATE|      BRANCH_STREET|BRANCH_ZIP|        LAST_UPDATED|
+-----------------+-----------+------------+------------+------------+-------------------+----------+--------------------+
|        Lakeville|          1|Example Bank|(610)123-456|          MN|       Bridle Court|     55044|2018-04-18T16:51:...|
|          Huntley|          2|Example Bank|(610)123-461|          IL|  Washington Street|     60142|2018-04-18T16:51:...|
|SouthRichmondHill|          3|Example Bank|(610)123-498|          NY|      Warren Street|     11419|2018-04-18T16:51:...|
|       Middleburg|          4|Example Bank|(610)123-466|          FL|   Cleveland Street|     32068|2018-04-18T16:51:...|
|    KingOfPrussia|          5|Example Bank|(610)123-484|          PA|        14th Street|     19406|2018-04-18T16:51:...|
|         Paters

In [7]:
df_branch.printSchema()

root
 |-- BRANCH_CITY: string (nullable = true)
 |-- BRANCH_CODE: long (nullable = true)
 |-- BRANCH_NAME: string (nullable = true)
 |-- BRANCH_PHONE: string (nullable = true)
 |-- BRANCH_STATE: string (nullable = true)
 |-- BRANCH_STREET: string (nullable = true)
 |-- BRANCH_ZIP: long (nullable = true)
 |-- LAST_UPDATED: string (nullable = true)



In [8]:
df_branch.write.format("jdbc") \
  .mode("append") \
  .option("url", "jdbc:mysql://localhost:3306/classicmodels") \
  .option("dbtable", "capstone_project.cdw_sapp_branch") \
  .option("user", "root") \
  .option("password", "password") \
  .save()

In [9]:
spark.stop()